In [10]:
import pandas as pd
import numpy as np
import joblib
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.cluster import KMeans

In [11]:
files = {
    "sekolah": "jrk_sekolah.csv", "ekonomi": "keberadaan_simpan_pinjam.csv",
    "olahraga": "lapangan_olahraga.csv", "sinyal": "sinyal_telepon.csv",
    "perikanan": "teknologi_perikanan.csv", "peternakan": "teknologi_peternakan.csv",
    "sampah": "tempat_sampah.csv", "nakes": "tenaga_kesehatan.csv"
}
dfs = {name: pd.read_csv(path) for name, path in files.items()}

In [12]:
key_cols = ['bps_kode_desa_kelurahan', 'tahun']
master_df = dfs['sekolah'][['bps_kode_desa_kelurahan', 'bps_nama_kabupaten_kota', 'bps_nama_desa_kelurahan', 'tahun', 'jarak_sd_terdekat']]

merge_map = {
    'ekonomi': 'keberadaan_bumdesa_keuangan_program_usaha_ekonomi_desa',
    'olahraga': 'jumlah_fasilitas_olahraga',
    'sinyal': 'status_sinyal_telepon_seluler',
    'perikanan': 'jumlah_alat_teknologi_tepat_guna_perikanan',
    'peternakan': 'jumlah_alat_teknologi_tepat_guna_peternakan',
    'sampah': 'ketersediaan_tempat_pembuangan_sampah',
    'nakes': 'jumlah_tenaga_kesehatan_lainnya'
}

for name, col in merge_map.items():
    subset = dfs[name][key_cols + [col]]
    master_df = pd.merge(master_df, subset, on=key_cols, how='inner')

master_df = master_df.dropna().reset_index(drop=True)

In [13]:
sinyal_map = {'SINYAL KUAT': 3, 'SINYAL LEMAH': 2, 'TIDAK ADA SINYAL': 1}
master_df['sinyal_score'] = master_df['status_sinyal_telepon_seluler'].map(sinyal_map).fillna(0)
master_df['bumdes_score'] = master_df['keberadaan_bumdesa_keuangan_program_usaha_ekonomi_desa'].apply(lambda x: 1 if x == 'ADA' else 0)
master_df['sampah_score'] = master_df['ketersediaan_tempat_pembuangan_sampah'].apply(lambda x: 1 if x == 'ADA' else 0)

In [14]:
num_cols = ['jarak_sd_terdekat', 'jumlah_fasilitas_olahraga', 'jumlah_alat_teknologi_perikanan', 'jumlah_alat_teknologi_peternakan', 'jumlah_nakes']
# Mapping nama kolom asli ke fitur numerik
features_numeric = ['jarak_sd_terdekat', 'jumlah_fasilitas_olahraga', 'jumlah_alat_teknologi_tepat_guna_perikanan',
                    'jumlah_alat_teknologi_tepat_guna_peternakan', 'jumlah_tenaga_kesehatan_lainnya']

X_log = master_df[features_numeric].apply(np.log1p)
X_cat = master_df[['bumdes_score', 'sinyal_score', 'sampah_score']]
X_final = pd.concat([X_log, X_cat], axis=1)

In [15]:
feature_order = ['jarak_sd_terdekat', 'bumdes_score', 'jumlah_fasilitas_olahraga', 'sinyal_score',
                 'jumlah_alat_teknologi_tepat_guna_perikanan', 'jumlah_alat_teknologi_tepat_guna_peternakan',
                 'sampah_score', 'jumlah_tenaga_kesehatan_lainnya']
X_final = X_final[feature_order]

In [16]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_final)

kmeans = KMeans(n_clusters=5, random_state=42, n_init=10)
master_df['cluster'] = kmeans.fit_predict(X_scaled)

In [17]:
status_map = {0: "Tertinggal", 1: "Maju", 2: "Mandiri (Ekonomi)", 3: "Sangat Tertinggal", 4: "Mandiri (Tata Kelola)"}
master_df['status_desa'] = master_df['cluster'].map(status_map)

In [18]:
master_df.to_csv("dataset_desa_final.csv", index=False)
joblib.dump(kmeans, 'model_status_desa.sav')
joblib.dump(scaler, 'scaler_desa.sav')
print("Master Dataset, Model, dan Scaler Berhasil Disimpan!")

OSError: [Errno 9] Bad file descriptor